In [1]:
#Homework 7
#Mackenzie James
#Due: 4/3/20

#collaborated with: 

In [9]:
# import modules
import numpy as np
import astropy.units as u
import astropy.constants as const

# import plotting modules
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# import Latex module so we can display the results with symbols
from IPython.display import Latex

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass

In [26]:
### Part 2: Defining All the Acceleration Terms ###


#creates a class that has a series of fucntions that will determine
#the acceleration M33 feels from M31, intergrate current position and 
#velocity forwards in time
class M33AnalyticOrbit:
    
    #initialize the class
    def __init__(self,filename):
    #inputs:
        #filename: takes the file that will will store the integrated orbit
        
        #Setting Gravitational Constant
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        #store output file name
        self.filename = filename
        
        ##relative COM position and velocity of M33 to M31 (from HW 4)
        #M33 data
        M33COM = CenterOfMass("M33_000.txt", 2)
        M33_COMP = M33COM.COM_P(0.1,4)
        M33_COMV = M33COM.COM_V(M33_COMP[0], M33_COMP[1], M33_COMP[2])
        #M31 data
        M31COM = CenterOfMass("M31_000.txt", 2)
        M31_COMP = M31COM.COM_P(0.1,2)
        M31_COMV = M31COM.COM_V(M31_COMP[0], M31_COMP[1], M31_COMP[2])
        #difference between M31 and M33, setting initial vector
        self.r0 = M31_COMP - M33_COMP
        self.v0 = M31_COMV - M33_COMV
        
        #scale length and masses for each component in M31
        self.rdisk = 5*u.kpc
        self.Mdisk = 0.12*1e12*u.Msun
        self.rbulge = 1*u.kpc
        self.Mbulge = 0.019*1e12*u.Msun
        self.rhalo = 60.2 #a for M31 from HW, check this value
        self.Mhalo = 1.921*1e12*u.Msun
        
    
    def HernquistAccel(self,M,ra,r0):
    #inputs:
        #M - total halo or bulge mass
        #ra - corresponding scale length
        #r0 - relative initial postion vector 
    #returns: 
        #the acceleration vector from a Hernquist potential
        
        #magnitude of positon vector
        rmag = np.sqrt((r0[0]**2)+(r0[1]**2)+(r0[2]**2))
        
        #acceleration vector
        Hern = -((G*M)/(rmag*((ra+rmag)**2)))*r0
        
        return Hern
    
    
    def MiyamotoNagaiAccel(self,M,rd,r0):
    #inputs:
        #M - Disk Mass
        #rd - self.rdisk value from __init__ function
        #r0 - relative initial postion vector
    #returns:
        #the acceleration vector for disk particles using MiyamotoNagai potential
        
        
        #magnitude of positon vector
        rmag = np.sqrt((r0[0]**2)+(r0[1]**2)+(r0[2]**2))
        
        #variables given in assignment above equation (3)
        R = np.sqrt((r0[0]**2)+(r0[1]**2))
        zd = self.rdisk/5.0
        B = rd + (np.sqrt((r0[2]**2)+(zd**2)))
        
        #the components for x and y are the same, so putting them in array as constant C
        # multiplied by the x and y component of r0
        
        C = -((G*M)/((R**2+B**2)**1.5))
        
        #now it will be that constant multiplied by an array with 1 for x and y
        # and adding the extra terms for z
        zterms = B/(np.sqrt(z**2+zd**2))
        MiyaN = C * np.array([1,1,zterms])
        
        return MiyaN
    
    
    #function that sums all acceleration vectors from each galaxy component
    def M31Accel(self):
    #return:
        #The 3D vector of the total acceleration
        
        AcclHalo = Hern(self.Mhalo,self.rhalo,self.r0)
        AcclBulge = Hern(self.Mbulge,self.rbulge,self.r0)
        AcclDisk = MiyaN(self.Mdisk,self.rdisk,self.r0)
        
        Acc = AcclHalo + AcclBulge + AcclDisk
        
        return Acc
    
### Part 3: Build an Integrator ###
    #function that is using the "Leap Frog" integration for the 
    # orbit of M33
    def LeapFrog(self,dt,r0,v0):
    #inputs:
        #dt - time interval for integration
        #r0 - starting postion vector for M33 COM position (relative to M31)
        #v0 - starting velocity vector for M33 (relative to M31)
    #return:
        #the new postion and velocity vectors
        
        #predicting position at the next halfstep
        rhalf = r0 + v0*(dt/2)
        
        #predict final velocity at next timestep using the acceleration at halfstep
        vnew = v0 + Acc*dt
        
        #predict final position using average of current and final velocity
        rnew = rhalf + vnew*(dt/2)
        
        return rnew, vnew
    
    #function to loop over the integrator to solve the equations of motion
    def OrbitIntegrator(self,t0,dt,tmax):
    #inputs:
        #t0 - starting time
        #dt- a time interval
        #tmax - final time
    #returns
        #future orbit of M33
        
        #initialze time t to the starting time
        t = t0
        
        #initialize an empty array from given parameters (in template)
        orbit = np.zeros([int((tmax/dt)+2),7])
        
        #initialize first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        
        #initialize a counter for the orbit
        i = 1
        
        #start integration
        while (t <= tmax):
            t = t + dt #advance the time by one timestep
            
            #store new time in first column of ith row
            orbit[i,0] = t
            
            #advance pos. and vel. by using LeapFrog
            #LeapFrog has two returns, so like position is LeapFrog[0] and veloctiy is LeapFrog [1]
            Pos = LeapFrog[0]
            Vel = LeapFrog[1]
            
            #store new postion vector
            orbit[i,1:4] = Pos
            #store new velocity vector
            orbit[i,4:7] = Vel
            
            #update counter i 
            i = i + 1
            
        #write data to a file
        np.savetotxt(self.filename, orbit, fmt= "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
            
            
        

In [27]:
Data = M33AnalyticOrbit("M33AnalyticOrbit.txt")